# Challenge Prompting

Resolver los siguientes ejercicios dejando el codigo con su ejecucion.

Importar las librerias necesarias y **correr las celdas para visualizar el resultado en cada ejercicio**.

In [6]:
## bloque importacion de librerias

import json
import ipywidgets as widgets
from IPython.display import display, clear_output

In [7]:
## bloque variables de entorno
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")


In [8]:
import cohere
## bloque conexion a Cohere

co = cohere.ClientV2()
# alternativa:
# co = cohere.ClientV2(api_key)

response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{"role": "user", "content": "hello world!"}],
)

print(response)



id='b29dc80d-328c-412f-ae95-662298625aee' finish_reason='COMPLETE' prompt=None message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='Hello! How can I help you today?')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=3.0, output_tokens=9.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=204.0, output_tokens=9.0)) logprobs=None


## Ejercicio 1

Extraccion de entidades

Utilizar el LLM para extraer las siguientes entidades del texto medico.

- Paciente:
    - Nombre
    - Edad
- Fecha de admisión
- Síntomas
- Diagnóstico
- Tratamiento recomendado

**Aclaracion:** 

La salida tiene que ser un **string con formato de tipo json**, el cual se convertira en un diccionario de Python.

Si la linea de conversion en test da error el ejercicio no esta completo.

In [4]:
# ejemplo 

# texto a analizar
"""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""


# respuesta del LLM
{
  "paciente": {
    "nombre": "María González",
    "edad": 45
  },
  "fecha_admision": "2023-08-05",
  "sintomas": [
    "fatiga crónica",
    "dolores musculares"
  ],
  "diagnostico": "fibromialgia",
  "tratamiento": [
    "fisioterapia",
    "medicamentos analgésicos"
  ]
}

{'paciente': {'nombre': 'María González', 'edad': 45},
 'fecha_admision': '2023-08-05',
 'sintomas': ['fatiga crónica', 'dolores musculares'],
 'diagnostico': 'fibromialgia',
 'tratamiento': ['fisioterapia', 'medicamentos analgésicos']}

In [5]:
text_to_analize = """Sofía López, de 28 años, ingresó al Hospital Infantil el 3 de abril de 2023 debido a fiebre alta y tos persistente./
Después de varias pruebas, se le diagnosticó neumonía. La pediatra responsable, Dra. Claudia Torres, indicó tratamiento con antibióticos y reposo./
La próxima evaluación será el 10 de abril."""

In [6]:
# your code here

prompt = f"""Extrae las siguientes entidades del texto médico y genera una respuesta en formato JSON. Si algún dato está ausente, omítelo en el JSON.
Entidades a extraer:
- Paciente:
    - Nombre
    - Edad
- Fecha de admisión
- Síntomas
- Diagnóstico
- Tratamiento recomendado

Ejemplo:
Texto:
"La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares. Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. La próxima consulta está programada para el 15 de septiembre."

Respuesta en JSON:
{{
  "paciente": {{
    "nombre": "María González",
    "edad": 45
  }},
  "fecha_admision": "2023-08-05",
  "sintomas": [
    "fatiga crónica",
    "dolores musculares"
  ],
  "diagnostico": "fibromialgia",
  "tratamiento": [
    "fisioterapia",
    "medicamentos analgésicos"
  ]
}}

Ahora, analiza el siguiente texto y extrae las entidades en el mismo formato JSON.

Texto:
"{text_to_analize}"
"""

response = co.generate(
        model="command-xlarge-nightly",
        prompt=prompt,
        temperature=0,
        max_tokens=300
    )
llm_response = response.generations[0].text.strip()

In [7]:
# test
import json
final_result = json.loads(llm_response)

#Verificar struct del json
def verifica_respuesta_llm(llm_response):
    try:
        # Intentar cargar la respuesta como JSON
        data = json.loads(llm_response)
        
        # Validar campos obligatorios
        if "paciente" in data:
            assert "nombre" in data["paciente"], "El nombre del paciente falta."
            assert "edad" in data["paciente"], "La edad del paciente falta."
        
        if "fecha_admision" in data:
            assert isinstance(data["fecha_admision"], str), "La fecha de admisión debe ser un string."
        
        if "sintomas" in data:
            assert isinstance(data["sintomas"], list), "Los síntomas deben ser una lista."
        
        if "diagnostico" in data:
            assert isinstance(data["diagnostico"], str), "El diagnóstico debe ser un string."
        
        if "tratamiento" in data:
            assert isinstance(data["tratamiento"], list), "El tratamiento debe ser una lista."
        
        print("La respuesta está en el formato correcto.")
        print(data)
    except json.JSONDecodeError:
        print("La respuesta no es un JSON válido.")
    except AssertionError as e:
        print(f"Error en la validación: {e}")

verifica_respuesta_llm(llm_response)

La respuesta está en el formato correcto.
{'paciente': {'nombre': 'Sofía López', 'edad': 28}, 'fecha_admision': '2023-04-03', 'sintomas': ['fiebre alta', 'tos persistente'], 'diagnostico': 'neumonía', 'tratamiento': ['antibióticos', 'reposo']}


## Ejercicio 2

Tenemos dos funciones en Python, una llamada *'add_contact'* y otra llamada *'get_information'*.

**Utilizar algun LLM que permita funtion calling** y desarrollar un codigo secuencial automatico que consiga:

Interpretar la consulta del usuario, identificar a que funcion llamar, luego llamarla (si es que aplica) y darle una respuesta final al usuario.  (usar function calling para esta solucion)

La entrada a dicho codigo es la consulta del usuario, a continuacion algunos ejemplos:

- "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
- "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
- "Cual es el Email de Juan Pérez.?"

Salidas esperadas de dichos ejemplos (variaran porque las genera el LLM):
-  El contacto fue anadido con exito
-  Se anadio el contacto
-  El email de juan perez es juanperez@mail.com

Link de ayuda: https://github.com/cohere-ai/notebooks/blob/main/notebooks/agents/Vanilla_Tool_Use_v2.ipynb

In [8]:
contacts = {
                        'Joaquin Lopez':{'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'},
                      'Flavio Oncativo':{'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}
}

In [9]:
def add_contact(name, phone, email):
    """
    Agrega un contacto al diccionario.
    Parámetros:
        name (str): Nombre del contacto.
        phone (str): Número de teléfono del contacto.
        email (str): Correo electrónico del contacto.
    Retorna:
        str: Mensaje confirmando la adición del contacto.
    """
    contacts[name] = {'phone': phone, 'email': email}
    return {"message":"Contacto agregado con exito."}

def get_information(name):
    """
    Recupera la información de un contacto.
    Parámetros:
        name (str): Nombre del contacto.
    Retorna:
        dict/str: Información del contacto o un mensaje si no existe.
    """
    if name in contacts:
        return contacts[name]
    else:
        return {"message":"Contacto no encontrado."}
    
functions_map = {
    "add_contact": add_contact,
    "get_information": get_information
}    

In [10]:
get_information("Joaquin Lopez")

{'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'}

In [11]:
print(contacts)

{'Joaquin Lopez': {'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'}, 'Flavio Oncativo': {'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}}


In [12]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_contact",
            "description": "Agrega un contacto al diccionario con su nombre, número de teléfono y correo electrónico.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Nombre completo del contacto a agregar."
                    },
                    "phone": {
                        "type": "string",
                        "description": "Número de teléfono del contacto a agregar."
                    },
                    "email": {
                        "type": "string",
                        "description": "Correo electrónico del contacto a agregar."
                    }
                },
                "required": ["name", "phone", "email"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_information",
            "description": "Recupera información de un contacto dado su nombre.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Nombre completo del contacto cuya información se desea recuperar."
                    }
                },
                "required": ["name"]
            }
        }
    }
]

preamble = """
## Tarea y Contexto
Tu objetivo es ayudar a las personas a interactuar contigo para resolver preguntas y realizar solicitudes relacionadas con la gestión de contactos. Puedes interpretar consultas en lenguaje natural, identificar la acción solicitada y ejecutar las funciones correspondientes para agregar contactos o recuperar información sobre ellos. Las funciones disponibles son:
1. `add_contact`: Agrega un contacto con nombre, número de teléfono y correo electrónico.
2. `get_information`: Recupera la información de un contacto dado su nombre.

Al interpretar una solicitud, debes:
- Analizar el mensaje del usuario para identificar la función adecuada.
- Llamar a la función con los parámetros correctos.
- Proporcionar una respuesta clara y completa al usuario.

## Guía de Estilo
A menos que el usuario solicite un estilo diferente, responde en oraciones completas utilizando una gramática y ortografía adecuadas. Sé educado, claro y directo al comunicar los resultados de las acciones realizadas.
"""

messages = ["Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com.",
"Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com.",
"Cual es el Email de Joaquin Lopez.?"]

In [13]:

def call_agent(message):
	messages=[{"role": "system", "content": preamble},
			{"role": "user", "content": message}]

	response = co.chat(
		model="command-r-plus",
		messages=messages,
		tools=tools
	)

	# print("El modelo recomienda ejecutar los siguientes metodos:\n")
	# print("Tool plan:")
	# print(response.message.tool_plan,"\n")
	# print("Tool calls:")
	# for tc in response.message.tool_calls:
	# 	print(f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}")
		
	# append the chat history
	messages.append({'role': 'assistant', 'tool_calls': response.message.tool_calls, 'tool_plan': response.message.tool_plan})
	
	tool_content = []
	# Iterate over the tool calls generated by the model
	for tc in response.message.tool_calls:
		# here is where you would call the tool recommended by the model, using the parameters recommended by the model
		tool_result= functions_map[tc.function.name](**json.loads(tc.function.arguments))
		# store the output in a list
		tool_content.append(json.dumps(tool_result))
		# append the chat history
		messages.append({'role': 'tool', 'tool_call_id': tc.id, 'content': tool_content}) 

	# print("Tool results:")
	# for result in tool_content:
	# 	print(json.dumps(json.loads(result), indent=2))
	
	response = co.chat(
    model="command-r-plus",
    messages=messages,
    tools=tools
	)

	return response.message.content[0].text

In [14]:
for message in messages:
    print(f"Mensaje: {message}")
    print(f"Respuesta: {call_agent(message)}\n")

Mensaje: Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com.
Respuesta: El contacto Juan Pérez ha sido agregado con éxito.

Mensaje: Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com.
Respuesta: He guardado a Lucía Gómez en tus contactos.

Mensaje: Cual es el Email de Joaquin Lopez.?
Respuesta: El correo electrónico de Joaquin Lopez es *Joacolocolopez@gmail.com*.



In [15]:
print(contacts)

{'Joaquin Lopez': {'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'}, 'Flavio Oncativo': {'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}, 'Juan Pérez': {'phone': '555-1234', 'email': 'juanperez@mail.com'}, 'Lucía Gómez': {'phone': '555-5678', 'email': 'lucia.gomez@gmail.com'}}


In [16]:
# TIPS
# Probar primero generando una funcion y llamarla, luego anadir la otra
# Plantearlo paso por paso en distintas celdas, analizar las salidas y las entradas, como identificamos a que funcion llamar?
# luego automatizar dentro de una sola celda


# Lo importante es entregar hasta donde lleguen, sea una funcion, las dos pero sin poder hacer el flujo automatico, lo que puedan, siempre y cuando este
# claro lo que se quizo hacer con comentarios.

## Ejercicio 3

Crear una funcion llamada "history_answer", que toma como parametro de entrada una pregunta sobre un contexto dado y la salida es la respuesta final del proceso impulsado por un LLM.

Dada una historia, el usuario podra hacer preguntas sobre la misma y el LLM debe responder siguiendo los siguientes lineamientos:

REQUISITOS DE LA RESPUESTA
- las respuestas deben ser en base a la historia
- ante la misma pregunta siempre debe responder de la misma manera.
- que responda en solo una oracion.
- el idioma que responde debe ser el mismo que con el que se pregunta (ingles, espanol, portugues).
- que agregue emojis en la oracion que resuman el contenido de la misma.
- que responda siempre en tercera persona.
- si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
- Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).

**Ayudin**: 
- No se limiten a usar 1 solo request al LLM, pueden dividirlo en partes para que por un lado se verifique el idioma, por otro lado se verifique si la pregunta tiene relacion con el contexto, etc

- Estructuren bien el prompt procurando separar instrucciones, contexto(historia) y pregunta del usuario.

- Recuerden usar el system message y user message.



In [2]:
historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""

In [3]:
def history_answer(pregunta, historia):
    # Paso 1: Identificar el idioma de la pregunta
    language_detection_prompt = [
        {"role": "system", "content": "Eres un asistente experto en detección de idiomas. Solo puedes responder con 'español', 'inglés' o 'portugués'."},
        {"role": "user", "content": f"¿En qué idioma está esta pregunta?\n\n'{pregunta}'"}
    ]
    response = co.chat(
        model='command-xlarge-nightly',
        messages=language_detection_prompt,
        max_tokens=10,
        temperature=0
    )

    #print(f"Idioma: {response.message.content[0].text.strip().lower()}")
    language = response.message.content[0].text.strip().lower()

    if language not in ["español", "inglés", "portugués"]:
        return "Lo siento no puedo detectar bien el idioma. Hakuna Matata! 🌈"

    # Paso 2: Verificar si la pregunta está relacionada con la historia
    relevance_check_prompt = [
        {"role": "system", "content": "Eres un asistente que determina si una pregunta está relacionada con una historia sin importar el idioma. Solo puedes responder 'sí' o 'no'."},
        {"role": "user", "content": f"""
        ¿La siguiente pregunta está relacionada con esta historia?

        Historia: "{historia}"
        Pregunta: "{pregunta}"
                """}
    ]
    response = co.chat(
        model='command-xlarge-nightly',
        messages=relevance_check_prompt,
        max_tokens=10,
        temperature=0
    )
    #print(f"Relevancia: {response.message.content[0].text.strip().lower()}")
    is_relevant = "sí" in response.message.content[0].text.strip().lower()

    if not is_relevant:
        return "Lo siento no puedo ayudarte con eso. Hakuna Matata! 🌈"

    # Paso 3: Generar la respuesta
    answer_prompt = [
        {"role": "system", "content": f"""
        Eres un asistente que responde preguntas basadas únicamente en una historia. Sigue estas reglas:
        - Responde solo en el idioma de la pregunta: {language}.
        - La respuesta debe basarse únicamente en la historia.
        - Usa una sola oración con emojis que resuman el contenido.
        - Responde siempre en tercera persona.
        - Agrega 'Hakuna Matata!' al final de la respuesta.
        """},
        {"role": "user", "content": f"""
            Historia: "{historia}"

        Pregunta: "{pregunta}"
        """}
    ]
    response = co.chat(
        model='command-xlarge-nightly',
        messages=answer_prompt,
        max_tokens=50,
        temperature=0.5
    )
    #print(f"Respuesta Previo validacion de idioma: {response.message.content[0].text.strip().lower()}")
    respuesta_al_usuario = response.message.content[0].text.strip()

    # Paso 4: Validar el idioma de la respuesta
    validation_prompt = [
        {"role": "system", "content": f"Eres un validador de idiomas. Verifica si el siguiente texto está en '{language}'. Responde con 'sí' o 'no'."},
        {"role": "user", "content": f"{respuesta_al_usuario}"}
    ]
    validation_response = co.chat(
        model='command-xlarge-nightly',
        messages=validation_prompt,
        max_tokens=10,
        temperature=0
    )
    #print(f"validacion de idioma: {validation_response.message.content[0].text.strip().lower()}")
    is_correct_language = validation_response.message.content[0].text.strip().lower() == "sí"

    if not is_correct_language:
        # Si el idioma no es correcto, fuerza la regeneración
        answer_prompt[0]["content"] += f"\n\nNOTA: Asegúrate de responder estrictamente en {language}."
        response = co.chat(
            model='command-xlarge-nightly',
            messages=answer_prompt,
            max_tokens=50,
            temperature=0.5
        )
        respuesta_al_usuario = response.message.content[0].text.strip()

    return respuesta_al_usuario


In [4]:
historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""

In [15]:
#Hago las preguntas por celdas porque hago muchas request y el trial me da un limite por minuto

pregunta = "¿Qué cambios experimentó Thomas tras regresar al feudo?"     

print(f"Pregunta: {pregunta}")
print(f"Respuesta: {history_answer(pregunta, historia)}")
print("")

Pregunta: ¿Qué cambios experimentó Thomas tras regresar al feudo?
Respuesta: 🗡️ Thomas regresó al feudo con el corazón endurecido, marcado por la guerra, y la mirada perdida en recuerdos de muerte y batalla. 🦅 Hakuna Matata!



In [43]:
pregunta = "What emotions did Thomas feel when he saw his companion fall on the battlefield?"

print(f"Pregunta: {pregunta}")
print(f"Respuesta: {history_answer(pregunta, historia)}")
print("")

Pregunta: What emotions did Thomas feel when he saw his companion fall on the battlefield?
Respuesta: 💔 Thomas felt a deep sense of loss and shock when he witnessed his comrade's tragic end, a harsh reminder of war's cruel reality. Hakuna Matata!



In [48]:
pregunta = "Quem é Thomas?"

print(f"Pregunta: {pregunta}")
print(f"Respuesta: {history_answer(pregunta, historia)}")
print("")

Pregunta: Quem é Thomas?
Respuesta: 🌾 Thomas é um jovem camponês que, aos 16 anos, troca a dura vida nos campos por uma lança, enfrentando a guerra e perdendo sua inocência no campo de batalha. Hakuna Matata!



In [46]:
pregunta = "¿Cuál es el mejor lugar para ir de vacaciones en invierno?"

print(f"Pregunta: {pregunta}")
print(f"Respuesta: {history_answer(pregunta, historia)}")
print("")

Pregunta: ¿Cuál es el mejor lugar para ir de vacaciones en invierno?
Respuesta: Lo siento no puedo ayudarte con eso. Hakuna Matata! 🌈



## Ejercicio 4

Crear un chatbot sencillo impulsado por un LLM. 

Dicho bot esta destinado a un usuario final y debe cumplir las siguientes **condiciones en sus respuestas**:

- Responder en no mas de 70 tokens.
- Responder de manera positiva, con un tono entusiasta.
- Responder con consejos útiles, como si fueras un tutor.

 
**Otras consideraciones**:

Respetar el formato de la interfaz provista por el ejercicio.

Ademas agregar al codigo propuesto un historial de conversaciones para que el bot pueda mantener el hilo de lo que se esta hablando. Para probar no usen mas de 3 conversaciones anidadas para no enviarle tantos tokens.

Dejar impreso en el notebook el historial de la conversacion.

In [213]:
# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()

system_message = """Eres un chatbot destinado a ayudar a los usuarios de manera amigable y entusiasta. 
Tus respuestas deben cumplir con las siguientes condiciones:
- Responde en no más de 50 tokens.
- Responde de manera positiva, con un tono entusiasta.
- Responde con consejos útiles, como si fueras un tutor.
Mantén un historial de la conversación para que puedas dar respuestas contextuales y útiles."""

chat_history = [{"role": "system", "content": system_message}]

# Función de respuesta simulada del chatbot
def chatbot_response(message):
    # Obtener la respuesta usando la función .chat de Cohere
    response = co.chat(
        model='command-xlarge-nightly',
        messages=chat_history,
        max_tokens=70  # Limitar la respuesta a 70 tokens
    )
    
    return response.message.content[0].text.strip().lower()

# Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        user_message = input_box.value.strip()
        
        if user_message:  # Solo procesar si el mensaje no está vacío
            # Mostrar el mensaje del usuario
            print(f"Tú: {user_message}")
            
            # Agregar el mensaje del usuario al historial
            chat_history.append({"role": "user", "content": user_message})
            
            # Obtener la respuesta del chatbot usando Cohere
            response = chatbot_response(user_message)
            
            # Mostrar la respuesta del chatbot
            print(f"Chatbot: {response}")
            
            # Agregar la respuesta del chatbot al historial
            chat_history.append({"role": "assistant", "content": response})
    
        
        # Limpiar el input después de enviar el mensaje
        input_box.value = ''

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)

Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()

In [214]:
# Mostrar el historial de la conversación
print("\nHistorial de conversación:")
for exchange in chat_history:
    print(exchange)


Historial de conversación:
{'role': 'system', 'content': 'Eres un chatbot destinado a ayudar a los usuarios de manera amigable y entusiasta. \nTus respuestas deben cumplir con las siguientes condiciones:\n- Responde en no más de 50 tokens.\n- Responde de manera positiva, con un tono entusiasta.\n- Responde con consejos útiles, como si fueras un tutor.\nMantén un historial de la conversación para que puedas dar respuestas contextuales y útiles.'}
{'role': 'user', 'content': 'Donde queda Argentina?'}
{'role': 'assistant', 'content': '¡hola! argentina es un país fascinante ubicado en el sur de américa del sur. limita con países como chile, bolivia, paraguay y brasil. ocupa una gran extensión de territorio, desde los andes en el oeste hasta el océano atlántico en el este. ¡es un país con una rica cultura y una naturaleza impresionante!'}
{'role': 'user', 'content': 'Cual es su capital?'}
{'role': 'assistant', 'content': '¡buenos aires! es una ciudad vibrante y llena de vida, considerada e

### RECOMENDACIONES GENERALES

No se confien probando con un par de respuestas y ya, hagan minimo 5 pruebas por ejercicio para asi tener mas chances de visualizar errores en la generacion del contenido.

Prueben combinar LLMs con programacion convencional para los casos que vean convenientes (decisiones if else, respuestas estaticas, etc)

Prueben con distintos modelos de Cohere, hay algunos optimizados para ciertas aplicaciones.